<a href="https://colab.research.google.com/github/AdiY2j/CS6910_Assignment2/blob/main/assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm

In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

In [3]:
torch.cuda.is_available()

True

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [ ]:

# Define the directory containing your dataset
dataset_dir = '/content/drive/MyDrive/inaturalist_12K'

data_aug = False

if data_aug :
    transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
else :
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

# Create a PyTorch dataset from the image folder
dataset = ImageFolder(dataset_dir, transform=transform)


validation_ratio = 0.2
class_labels = [label for _, label in dataset]

# Create a dictionary to store indices for each class
class_indices = defaultdict(list)
for idx, label in enumerate(class_labels):
    class_indices[label].append(idx)

# Initialize lists to store training and validation indices
train_indices = []
val_indices = []

# Split each class into training and validation indices
for label, indices in class_indices.items():
    num_samples = len(indices)
    num_validation_samples = int(validation_ratio * num_samples)
    np.random.shuffle(indices)  # Shuffle indices for random selection
    train_indices.extend(indices[num_validation_samples:])
    val_indices.extend(indices[:num_validation_samples])

# Create SubsetRandomSampler for training and validation sets
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

# Create dataloaders for training and validation sets
batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)
